In [1]:
import json
from collections import Counter, defaultdict, deque
import gc

import numpy as np
import pandas as pd
import glob
import time
from tqdm import tqdm

from sklearn import model_selection, metrics
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import seaborn as sn

import lightgbm as lgb
from scipy.stats import skew

import feather
import pickle
import h5py

from concurrent import futures
#import riiideducation

In [2]:
# lyaka1 = np.load("./ssakt15_valid_preds.npy")
# lyaka2 = np.load("./ssakt31_valid_preds.npy")
# sakami["lyaka1"] = lyaka1
# sakami["lyaka2"] = lyaka2

In [3]:
owruby =  pd.read_csv("./owruby/owruby_new.csv")

In [4]:
owruby.head(2)

,row_id,answered_correctly,label
0,30134522,0.852429,0
1,92007055,0.984587,1


In [5]:
sakami_rows = owruby[["row_id"]].sort_values(by="row_id")

In [6]:
sakami_rows["sakami169"] = np.load("./sakami/sakami169_new.npy")
sakami_rows["sakami172"] = np.load("./sakami/sakami172_new.npy")
sakami_rows["sakami174"] = np.load("./sakami/sakami174_new.npy")
sakami_rows["sakami175"] = np.load("./sakami/sakami175_new.npy")
sakami_rows["sakami177"] = np.load("./sakami/sakami177_new.npy")
sakami_rows["sakami180"] = np.load("./sakami/sakami180_new.npy")
sakami_rows["sakami182"] = np.load("./sakami/sakami182_new.npy")

In [7]:
sakami_rows.head(2)

,row_id,sakami169,sakami172,sakami174,sakami175,sakami177,sakami180,sakami182
3162736,46,0.835227,0.826471,0.836535,0.837373,0.825483,0.841123,0.829501
3162888,47,0.338686,0.393997,0.317133,0.310963,0.318466,0.369737,0.390671


In [8]:
merged = pd.merge(owruby, sakami_rows, on="row_id", how="inner")
merged.head(2)

,row_id,answered_correctly,label,sakami169,sakami172,sakami174,sakami175,sakami177,sakami180,sakami182
0,30134522,0.852429,0,0.863962,0.896593,0.872145,0.847413,0.862121,0.883331,0.887582
1,92007055,0.984587,1,0.972127,0.958629,0.950926,0.936254,0.935615,0.980715,0.979626


In [10]:
pocket = pd.read_csv("./pred_0105_pre.csv")
pocket2 = pd.read_csv("../pred_0105_nonn.csv")
pocket3 = pd.read_csv("../pred_0105_nouni.csv")
pocket4 = pd.read_csv("../pred_0105_2.csv")

In [11]:
pocket.head()

,0
0,0.837914
1,0.930254
2,0.525241
3,0.147935
4,0.159508


In [12]:
merged["pocket"] = pocket
merged["pocket2"] = pocket2
merged["pocket3"] = pocket3
merged["pocket4"] = pocket4

In [13]:
merged["owruby"] = merged["answered_correctly"]
merged.drop(columns="answered_correctly", inplace=True)

In [14]:
merged.head(2)

,row_id,label,sakami169,sakami172,sakami174,sakami175,sakami177,sakami180,sakami182,pocket,pocket2,pocket3,pocket4,owruby
0,30134522,0,0.863962,0.896593,0.872145,0.847413,0.862121,0.883331,0.887582,0.837914,0.845037,0.855801,0.850908,0.852429
1,92007055,1,0.972127,0.958629,0.950926,0.936254,0.935615,0.980715,0.979626,0.930254,0.911940,0.914832,0.916506,0.984587


In [15]:
merged.columns

Index(['row_id', 'label', 'sakami169', 'sakami172', 'sakami174', 'sakami175',
       'sakami177', 'sakami180', 'sakami182', 'pocket', 'pocket2', 'pocket3',
       'pocket4', 'owruby'],
      dtype='object')

In [16]:
valid_feats = pd.read_feather("../valid_0105.feather")

In [17]:
valid_feats.head(2)

,index,row_id,user_id,ac,et,q_ac_mean,q_ac_cnt,q_et_mean,q_et_cnt,q_et_std,...,u_td_p4p5,u_td_p5p6,u_td_p6p7,u_td_p7p8,u_td_p8p9,u_td_p9p10,ut_ac_mean,ut_ac_mean2,u_ok_ts,u_ng_ts
0,96146750,30134522,646513198,0,58400.0,0.757407,19238,49663.562500,19238,31223.854926,...,18725.0,26927.0,21909.0,43204.0,36228.0,25611.0,1.0,1.0,-393316.0,-16143745.0
1,96146751,92007055,1952358543,1,30000.0,0.810682,4606,24951.654297,4606,22371.913493,...,1942593.0,3341194.0,23347.0,29095.0,115404.0,999108.0,NaN,NaN,-62158.0,-20447.0


In [18]:
valid_feats.shape

(5000000, 83)

In [19]:
merged = pd.merge(merged, valid_feats, on="row_id", how="inner")

In [20]:
merged.head(2)

,row_id,label,sakami169,sakami172,sakami174,sakami175,sakami177,sakami180,sakami182,pocket,...,u_td_p4p5,u_td_p5p6,u_td_p6p7,u_td_p7p8,u_td_p8p9,u_td_p9p10,ut_ac_mean,ut_ac_mean2,u_ok_ts,u_ng_ts
0,30134522,0,0.863962,0.896593,0.872145,0.847413,0.862121,0.883331,0.887582,0.837914,...,18725.0,26927.0,21909.0,43204.0,36228.0,25611.0,1.0,1.0,-393316.0,-16143745.0
1,92007055,1,0.972127,0.958629,0.950926,0.936254,0.935615,0.980715,0.979626,0.930254,...,1942593.0,3341194.0,23347.0,29095.0,115404.0,999108.0,NaN,NaN,-62158.0,-20447.0


In [21]:
class SingleLgb:
    def __init__(self, seed=99, dry_run=False):
        self.train_param = self.get_param()
        if dry_run:
            self.num_rounds = 100
        else:
            self.num_rounds = 1100

    def do_train_direct(self, x_train, x_test, y_train, y_test):
        lgb_train = lgb.Dataset(x_train, y_train)
        lgb_eval = lgb.Dataset(x_test, y_test)

        # print('Start training...')
        model = lgb.train(self.train_param,
                          lgb_train,
                          valid_sets=[lgb_eval],
                          verbose_eval=100,
                          num_boost_round=self.num_rounds,
                          early_stopping_rounds=100,
                          #categorical_feature=[]
                         )
        # print('End training...')
        return model

    @staticmethod
    def show_feature_importance(model, filename=None):
        fi = pd.DataFrame({
            "name": model.feature_name(),
            "importance_split": model.feature_importance(importance_type="split").astype(int),
            "importance_gain": model.feature_importance(importance_type="gain").astype(int),
        })
        fi = fi.sort_values(by="importance_gain", ascending=False)
        #print(fi)
        with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
            print(fi)

    @staticmethod
    def get_param():
        return {
            'num_leaves': 127,
            'min_data_in_leaf': 100,
            'objective': 'binary',
            #'metric': 'auc',
            'metric': 'binary_logloss',
            'max_depth': -1,
            'learning_rate': 0.02,
            "boosting": "gbdt",
            "feature_fraction": 0.9,
            "verbosity": -1,
            "random_state": 81,
        }
    
class SingleTrainer:
    def __init__(self, pred_col, dry_run=False):
        self.pred_col = pred_col
        self.target_col = "ac"
        self.dry_run = dry_run
        self.val_size = 2*1000*1000
        
#     def do_kfold(self, df):
#         X = df[self.pred_col]
#         y = df[self.target_col]
        
#         models, scores = list(), list()
#         KFold(n_splits=5, random_state=None, shuffle=False)
#         for train_index, test_index in kf.split(X):
#             print("---------")
#             print("fold=", fold)
#             f, c = fold, self.val_size
#             val_s, val_e = -c-f*c, len(df)-f*c
#             train_idx = -c-f*c
#             X_train, X_val = X.iloc[:train_idx], X.iloc[val_s:val_e]
#             y_train, y_val = y.iloc[:train_idx], y.iloc[val_s:val_e]
#             print(X_train.shape, X_val.shape)
            
#             lgbm = SingleLgb(seed=99, dry_run=self.dry_run)
#             model = lgbm.do_train_direct(X_train, X_val, y_train, y_val)
#             score = model.best_score["valid_0"]["binary_logloss"]
#             pred = model.predict(X_val)
#             score = metrics.roc_auc_score(y_val, pred)
#             print("AUC=", score)
#             if fold == 0:
#                 lgbm.show_feature_importance(model)
#             models.append(model)
#             scores.append(score)
#             break
#         return models, np.mean(scores)
#             X_train, X_test = X[train_index], X[test_index]
#             y_train, y_test = y[train_index], y[test_index]

    def train_model(self, df):
        X = df[self.pred_col]
        y = df[self.target_col]
        
        models, scores = list(), list()
        for fold in range(4):
            print("---------")
            print("fold=", fold)
            f, c = fold, self.val_size
            val_s, val_e = -c-f*c, len(df)-f*c
            train_idx = -c-f*c
            X_train, X_val = X.iloc[:train_idx], X.iloc[val_s:val_e]
            y_train, y_val = y.iloc[:train_idx], y.iloc[val_s:val_e]
            print(X_train.shape, X_val.shape)
            
            lgbm = SingleLgb(seed=99, dry_run=self.dry_run)
            model = lgbm.do_train_direct(X_train, X_val, y_train, y_val)
            score = model.best_score["valid_0"]["binary_logloss"]
            pred = model.predict(X_val)
            score = metrics.roc_auc_score(y_val, pred)
            print("AUC=", score)
            if fold == 0:
                lgbm.show_feature_importance(model)
            models.append(model)
            scores.append(score)
            break
        return models, np.mean(scores)

In [22]:
pred_col = [
    "et",  "q_ac_cnt",  'b_ac_mean', 'b_ac_cnt', "q_ac_mean",
    "u_ac_mean", "u_cnt", "u_qm_mean", #"u_td", #"uc_td",
    "u_ac_cnt", #"u_et_cnt"
    'u_ok_qm_mean', 'u_ng_qm_mean',
    'q_et_mean', 'q_et_cnt', 'q_et_std',
   "up_ac_cnt", "up_ac_mean",
    "u_td", 
    "q_ng_uac_mean","q_ok_uac_mean", "q_ok_uac_std", "q_ng_uac_std",
    "correct_answer",
    "u_ac_mean20",
    "ulr_ac_mean", #"ulr_ac_cnt",  "ub_cnt",
    "ut_ac_mean","ut_ac_mean2",
    "uca_ac_mean", 
    "ul_td", "u_td_wl", 
    #"q_pqhe_true_uac_mean", "q_pqhe_true_uac_std", "q_pqhe_false_uac_mean", "q_pqhe_false_uac_std",
    #"pqhe",
    "uc_td", "uc_ac_mean", "uc_ac_cnt", 
    #"u_td_final", #"ub_td_final"
    'u_td_tp1','u_td_p1p2', 'u_td_p2p3', 'u_td_p3p4', 'u_td_p4p5',
    'u_td_p5p6', 'u_td_p6p7', 'u_td_p7p8', 'u_td_p8p9', 'u_td_p9p10',
    "uac_prev1", "uac_prev2",
    "elo_beta", "u_elo_theta", 
    "u_rate",
]
nn_col = [f"nn_svd{i}" for i in range(20)]
pred_col += nn_col
print(pred_col)

['et', 'q_ac_cnt', 'b_ac_mean', 'b_ac_cnt', 'q_ac_mean', 'u_ac_mean', 'u_cnt', 'u_qm_mean', 'u_ac_cnt', 'u_ok_qm_mean', 'u_ng_qm_mean', 'q_et_mean', 'q_et_cnt', 'q_et_std', 'up_ac_cnt', 'up_ac_mean', 'u_td', 'q_ng_uac_mean', 'q_ok_uac_mean', 'q_ok_uac_std', 'q_ng_uac_std', 'correct_answer', 'u_ac_mean20', 'ulr_ac_mean', 'ut_ac_mean', 'ut_ac_mean2', 'uca_ac_mean', 'ul_td', 'u_td_wl', 'uc_td', 'uc_ac_mean', 'uc_ac_cnt', 'u_td_tp1', 'u_td_p1p2', 'u_td_p2p3', 'u_td_p3p4', 'u_td_p4p5', 'u_td_p5p6', 'u_td_p6p7', 'u_td_p7p8', 'u_td_p8p9', 'u_td_p9p10', 'uac_prev1', 'uac_prev2', 'elo_beta', 'u_elo_theta', 'u_rate', 'nn_svd0', 'nn_svd1', 'nn_svd2', 'nn_svd3', 'nn_svd4', 'nn_svd5', 'nn_svd6', 'nn_svd7', 'nn_svd8', 'nn_svd9', 'nn_svd10', 'nn_svd11', 'nn_svd12', 'nn_svd13', 'nn_svd14', 'nn_svd15', 'nn_svd16', 'nn_svd17', 'nn_svd18', 'nn_svd19']


In [44]:
pred_col = [
    'sakami169', 'sakami172', 'sakami177',
    'sakami174', 'sakami175', 'sakami180', 'sakami182', 
    'owruby', 'pocket4', 
    "uc_td", "uc_ac_mean", "uc_ac_cnt", 
    "ul_td", "u_td", "u_td_p1p2", 
    "ut_ac_mean", "ut_ac_mean2",
    "up_ac_mean",
    "et", "q_et_mean", "q_et_std",
    "u_elo_theta","q_ac_mean",  #"ulr_elo_mean", "up_elo_mean",
    "up_ac_cnt", "u_cnt", "q_ac_cnt"
    
#         "et",  "q_ac_cnt",  'b_ac_mean', 'b_ac_cnt', "q_ac_mean",
#     "u_ac_mean", "u_cnt", "u_qm_mean", #"u_td", #"uc_td",
#     "u_ac_cnt", #"u_et_cnt"
#     'u_ok_qm_mean', 'u_ng_qm_mean',
#     'q_et_mean', 'q_et_cnt', 'q_et_std',
#    "up_ac_cnt", "up_ac_mean",
#     "u_td", 
#     "q_ng_uac_mean","q_ok_uac_mean", "q_ok_uac_std", "q_ng_uac_std",
#     "correct_answer",
#     "u_ac_mean20",
#     "ulr_ac_mean", #"ulr_ac_cnt",  "ub_cnt",
#     "ut_ac_mean","ut_ac_mean2",
#     "uca_ac_mean", 
#     "ul_td", "u_td_wl", 
#     #"q_pqhe_true_uac_mean", "q_pqhe_true_uac_std", "q_pqhe_false_uac_mean", "q_pqhe_false_uac_std",
#     #"pqhe",
#     "uc_td", "uc_ac_mean", "uc_ac_cnt", 
#     #"u_td_final", #"ub_td_final"
#     'u_td_tp1','u_td_p1p2', 'u_td_p2p3', 'u_td_p3p4', 'u_td_p4p5',
#     'u_td_p5p6', 'u_td_p6p7', 'u_td_p7p8', 'u_td_p8p9', 'u_td_p9p10',
#     "uac_prev1", "uac_prev2",
#     "elo_beta", "u_elo_theta", 
#     "u_rate",
]

In [45]:
#sorted_df = merged.sort_values(by="row_id")

In [47]:
trainer = SingleTrainer(pred_col, dry_run=False)
models, score = trainer.train_model(merged)

---------
fold= 0
(3000000, 26) (2000000, 26)
Training until validation scores don't improve for 100 rounds
[100]	valid_0's binary_logloss: 0.520859
[200]	valid_0's binary_logloss: 0.514638
[300]	valid_0's binary_logloss: 0.514231
[400]	valid_0's binary_logloss: 0.514158
[500]	valid_0's binary_logloss: 0.514116
[600]	valid_0's binary_logloss: 0.514099
[700]	valid_0's binary_logloss: 0.514078
[800]	valid_0's binary_logloss: 0.514068
[900]	valid_0's binary_logloss: 0.514066
Early stopping, best iteration is:
[855]	valid_0's binary_logloss: 0.514063
AUC= 0.8118673649328597
           name  importance_split  importance_gain
1     sakami172              4897         15468580
0     sakami169              4488          4580422
7        owruby              6960          3440765
8       pocket4              7805           350774
2     sakami177              4158           258232
6     sakami182              3924           122005
12        ul_td              5894            80654
24        u_cnt

In [48]:
# new valid 3m-2m, 20feats: 

In [49]:
models[0].save_model("./ensemble_model_0105.lgb")

In [ ]:
# before lyaka 8140
# after sakami1 8141
# after lyaka12 8142
# only lyaka12, no sakami1 0.81419: model0101
# only lyaka2 0.81415

# valid 

In [55]:
# base feats
#Early stopping, best iteration is:
#[295]	valid_0's binary_logloss: 0.517508
#AUC= 0.7967711921542647

# with owruby, sakami
# [73]	valid_0's binary_logloss: 0.499981
#AUC= 0.812631130956799

# add pocket1223
#AUC= 0.8130389687940041

#just 3
#AUC= 0.8119216727001559

# just3 + top feats
#0.81279

In [58]:
class SeedLgb:
    def __init__(self, seed=99, dry_run=False):
        self.train_param = self.get_param(seed)
        if dry_run:
            self.num_rounds = 100
        else:
            self.num_rounds = 1100

    def do_train_direct(self, x_train, y_train):
        lgb_train = lgb.Dataset(x_train, y_train)

        # print('Start training...')
        model = lgb.train(self.train_param,
                          lgb_train,
                          valid_sets=None,
                          verbose_eval=100,
                          num_boost_round=self.num_rounds,
                          #categorical_feature=[]
                         )
        # print('End training...')
        return model

    @staticmethod
    def show_feature_importance(model, filename=None):
        fi = pd.DataFrame({
            "name": model.feature_name(),
            "importance_split": model.feature_importance(importance_type="split").astype(int),
            "importance_gain": model.feature_importance(importance_type="gain").astype(int),
        })
        fi = fi.sort_values(by="importance_gain", ascending=False)
        #print(fi)
        with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
            print(fi)

    @staticmethod
    def get_param(seed):
        return {
            'num_leaves': 127,
            'min_data_in_leaf': 100,
            'objective': 'binary',
            #'metric': 'auc',
            'metric': 'binary_logloss',
            'max_depth': -1,
            'learning_rate': 0.02,
            "boosting": "gbdt",
            "feature_fraction": 0.9,
            "verbosity": -1,
            "random_state": seed,
        }
    
class SeedTrainer:
    def __init__(self, pred_col, dry_run=False):
        self.pred_col = pred_col
        self.target_col = "ac"
        self.dry_run = dry_run

    def train_model(self, df):
        X = df[self.pred_col]
        y = df[self.target_col]
        
        models, scores = list(), list()
        for fold in range(4):
            print("---------")
            print("fold=", fold)
            
            lgbm = SeedLgb(seed=fold, dry_run=self.dry_run)
            model = lgbm.do_train_direct(X, y)
            #score = model.best_score["valid_0"]["binary_logloss"]
            #pred = model.predict(X_val)
            #score = metrics.roc_auc_score(y_val, pred)
            #print("AUC=", score)
            if fold == 0:
                lgbm.show_feature_importance(model)
            models.append(model)
            #scores.append(score)
        return models, scores

In [59]:
trainer = SeedTrainer(pred_col, dry_run=False)
models, score = trainer.train_model(merged)

---------
fold= 0
           name  importance_split  importance_gain
1       sakami2              9218         21663621
2       owruby2             10160         16689718
0   pocket_1226             11127           640644
3        lyaka2              7528           252855
4         uc_td              6046           140199
6         ul_td              9285           131655
7          u_td              7975            96863
5    uc_ac_mean              2511            93665
19        u_cnt              7030            93439
16    q_ac_mean              7188            89085
15  u_elo_theta              7249            81063
8     u_td_p1p2              6702            73352
20     q_ac_cnt              6421            68977
11   up_ac_mean              6620            68750
13    q_et_mean              5564            64248
17    up_ac_cnt              5850            62281
9    ut_ac_mean              5259            61034
12           et              5675            60832
10  ut_ac_mea

In [60]:
for i in range(4):
    models[i].save_model(f"./ensemble_model_0101_all{i}.lgb")

In [22]:
metrics.roc_auc_score(merged2["label"], merged2["ranked_3avg"])

0.8115313781174149

In [17]:
for power in [1.5, 1.75, 2, 2.25, 2.5, 3, 3.5, 4]:
    merged2["power_mean"] = (merged2["sakami"]**power + merged2["owruby"]**power + merged2["pocket_1223"]**power)
    score = metrics.roc_auc_score(merged2["label"], merged2["power_mean"])
    print(power, score)

1.5 0.8115757793963951
1.75 0.8115617807361065
2 0.8115436387419797
2.25 0.8115220983411143
2.5 0.8114978412488558
3 0.811442840131525
3.5 0.8113818106061845
4 0.8113171371978005
